# Understanding how to control memory allocation
##### (Of functions that are called lots of times and generate arrays everytime they are called)

This notebook compares two versions of a function:  **```compute_grad```** and **```compute_grad_with_dot!```**.

The idea was to have a type that has "placeholders" for the quantities that are computed inside ```compute_grad``` (sampling quantities, as well as V_hat, H_hat) to avoid allocating memory at every update of the parameters of the model (at every call to  ```compute_grad```).


In [ ]:
à

In [1]:
# Import Distributions to generate random numbers W matrix of the RBM
using Distributions
using MNIST
using BenchmarkTools
using Combinatorics

In [2]:
X_train, y_train = MNIST.traindata()
X_test, y_test = MNIST.testdata()

T = Float32
X_train = Array{T}((X_train - minimum(X_train))/(maximum(X_train) - minimum(X_train)))
y_train = Array{T}(y_train)
X_test = Array{T}(X_test - minimum(X_test))/(maximum(X_test) - minimum(X_test)) 
y_test = Array{T}(y_test);

##  Basic types

In [3]:
include("./src/RBM_models.jl")

RBM_Models

In [4]:
import RBM_Models: initialize_CDK, initialize_RBM, partial_fit!, fit!

In [5]:
rbm = initialize_RBM(784, 20, 0.01, Float32);
cdk = initialize_CDK(rbm, 2, 500);

### Benchmark RBM

In [6]:
#@benchmark partial_fit!(rbm,  X_train[:,1:500]  , 0.1, cdk)
@benchmark partial_fit!(rbm,  view(X_train,:,1:500)  , 0.1, cdk)

BenchmarkTools.Trial: 
  memory estimate:  1.54 MiB
  allocs estimate:  91
  --------------
  minimum time:     15.905 ms (0.00% GC)
  median time:      16.286 ms (0.00% GC)
  mean time:        16.473 ms (0.81% GC)
  maximum time:     21.329 ms (9.86% GC)
  --------------
  samples:          302
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [ ]:
@time fit!(rbm, X_train, 500, 10, 0.1, true, cdk)

In [ ]:
@benchmark rbm.W*(@view X_train[:,1:500])

In [ ]:
@benchmark rbm.W * X_train[:,1:500]

In [ ]:
@time RBM_Models.update_params!(rbm, cdk, 0.01);

In [ ]:
@time RBM_Models.compute_grad!(rbm,X_train[:,1:500] , cdk);

In [ ]:
X = rand(784, 500)
Vhat = rand(784, 500);

In [ ]:
size(sum((X .- Vhat), 2))

In [ ]:
size( Array{Float32}(zeros(size(X,1))))

In [ ]:
@time sum((X .- Vhat), 2);

In [ ]:
function grad_bias(X::Matrix, Vhat::Matrix, batch_size)
    suma = zeros(size(X,1))
    aux = 0 
    @inbounds for i in 1:size(X,1)
        @simd for j in 1:size(X,2)
            suma[i] += X[i,j] - Vhat[i,j]
        end
    end
    return suma./batch_size
end

In [ ]:
size(X)

In [ ]:
@time grad_bias(X,Vhat);

In [ ]:
@time squeeze(sum((X .- Vhat), 2),2);

In [ ]:
r1 = sum((X - Vhat), 2); 
r2 = grad_bias(X,Vhat);

sum(r1), sum(r2)

In [ ]:
x = rand(1000,1000);

In [ ]:
@time x .= zeros(1000,1000);

In [ ]:
@benchmark partial_fit!(rbm,  X_train[:,1:500]  , 0.1, cdk)

In [ ]:
@time fit!(rbm, X_train, 500, 10, 0.1, true, cdk)

In [ ]:
@benchmark partial_fit!(rbm,  X_train[:,1:500]  , 0.1, cdk)

In [ ]:
@time fit!(rbm, X_train, 500, 10, 0.1, true, cdk)

## Benchmark RBM_Blas

In [ ]:
aux_bias = zeros(784);

In [ ]:
@time sum!(aux_bias, X_train[:,1:500]);

In [ ]:
#@time aux_bias .= sum(X_train[:,1:500],2)

In [ ]:
#Xbatch = X_train[:,1:100];

In [ ]:
?BLAS.gemm!


    gemm!(tA, tB, alpha, A, B, beta, C)

Update C as alpha * A * B + beta*C or the other three variants according to tA (transpose A) and tB. Returns the updated C.
Note: here, alpha and beta must be float type scalars. A, B and C are all matrices. It's up to you to make sure the matrix dimensions match.

Thus, the tA and tB parameters refer to whether you want to apply the transpose operation to A or to B before multiplying. Note that this will cost you some computation time and allocations - the transpose isn't free. (thus, if you were going to apply the multiplication many times, each time with the same transpose specification, you'd be better off storing your matrix as the transposed version from the beginning). Select N for no transpose, T for transpose. You must select one or the other.

The difference between **```gemm!()```** and **```gemv!()```** is that for **```gemm!()```** you already need to have allocated the matrix C. The ! is a "modify in place" signal. Consider the following illustration of their different uses:

    A = rand(5,5)
    B = rand(5,5)
    C = Array(Float64, 5, 5)

    BLAS.gemm!('N', 'T', 1.0, A, B, 0.0, C)
    D = BLAS.gemm('N', 'T', 1.0, A, B)


julia> C == D
true

Each of these, in essence, perform the calculation C = A * B'. (Technically, gemm!() performs C = (0.0)*C + (1.0)*A * B'.)

Thus, the syntax for the modify in place **```gemm!()```** is a bit unusual in some respects (unless you've already worked with a language like C in which case it seems very intuitive). You don't have the explicit = sign like you frequently do when calling functions in assigning values in a high level object oriented language like Julia.

As the illustration above shows, the outcome of **```gemm!()```** and **```gemm()```** in this case is identical, even though the syntax and procedure to achieve that outcome is a bit different. Practically speaking, however, performance differences between the two can be significant, depending on your use case. In particular, if you are going to be performing that multiplication operation many times, replacing/updating the value of C each time, then gemm!() can be a decent bit quicker because you don't need to keep re-allocating new memory each time, which does have time costs, both in the initial memory allocation and then in the garbage collection later on.



In [ ]:
#BLAS.gemm!('N','N',Xbatch,Xbatch[:,1],Xbatch[:,1])

- http://stackoverflow.com/questions/38481674/julia-blas-gemm-parameters

In [ ]:
W = rand(200,784)
X = rand(784,100)
C = Array(Float64, 200);

H_aux = rand(200,100);

In [ ]:
expand(:(W*X' .+ X))

In [ ]:
@time BLAS.gemm('N', 'N', 1.0, W, X);

In [ ]:
@time BLAS.gemm!('N', 'N', 1.0, W, X,0.0,H_aux);

In [ ]:
@time  W*X;

In [ ]:
size(W*X)

In [ ]:
BLAS.

In [ ]:
function A_mul_B_plus_C!(C,A,B)
    BLAS.gemm!('N', 'N', 1.0, A, B, 0.0, C);
end

In [ ]:
@benchmark A_mul_B_plus_C!(C,A,B)

In [ ]:
@benchmark C .= A_mul_B!(C,A,B) .+ 0*C 

In [ ]:
@time BLAS.gemm!('N', 'N', 1.0, A, B, 0.0, C);

In [ ]:
@benchmark BLAS.gemm!('N', 'N', 1.0, A, B, 0.0, C)

In [ ]:
@benchmark A_mul_B!(C,A,B) 

In [ ]:
#@benchmark C .= A_mul_B!(C,A,B) .+ 0*C 

In [ ]:
@which A_mul_Bt

In [ ]:
function suma_cols(X)
    n_rows, n_cols = size(X)
    suma = Array{Float32}(zeros(n_rows))
    
    @simd for i in n_rows
        for j in n_cols
            suma[j] .+= X[i,j]
        end
    end
    return suma
end

In [ ]:
@time aux .= suma_cols(X_train[:,1:500]);

In [ ]:
sum(aux)

In [ ]:
size(X_train[:,1:500])

In [ ]:
size(sum(X_train[:,1:500],2))

In [ ]:
@time sum(X_train[:,1:500],2);

In [ ]:
@benchmark sum!(aux_bias,X_train[:,1:500])

In [ ]:
@benchmark aux_bias .= sum(X_train[:,1:500],2)

### Benchmark RBM

In [ ]:
@time partial_fit!(rbm,  X_train[:,1:500], 0.1, cdk);

In [ ]:
@time partial_fit!(rbm,  X_train[:,1:500], 0.1, cdk);

In [ ]:
@benchmark partial_fit!(rbm,  X_train[:,1:500]  , 0.1, cdk)

In [ ]:
@time fit!(rbm, X_train, 500,1,0.01,true,cdk)

In [ ]:
# function partial_fit!(rbm::RBM, X::Matrix, K::Integer, lr::Real, optimizer::CDK)
@benchmark partial_fit!(rbm,  X_train[:,1:500]  , 0.1, cdk)

In [ ]:
# function partial_fit!(rbm::RBM, X::Matrix, K::Integer, lr::Real, optimizer::CDK)
@benchmark partial_fit!(rbm, X_train[:,1:500], 0.1, cdk)

In [ ]:
# function partial_fit!(rbm::RBM, X::Matrix, K::Integer, lr::Real, optimizer::CDK)
@benchmark partial_fit!(rbm, X_train[:,1:500], 0.1, cdk)

In [ ]:
# function partial_fit!(rbm::RBM, X::Matrix, K::Integer, lr::Real, optimizer::CDK)
@benchmark partial_fit!(rbm, X_train[:,1:500], 0.1, cdk)